In [125]:
import pandas as pd
import numpy as np
import dateutil.easter
from prophet import Prophet

In [110]:
df_national = pd.read_csv(
    '../data/processed/data_processed_v3_National_NoAcum_Total.csv')
df_national.head()

,Year,Week,Date,Epi_Year,total_cases
0,2014,2,2014-01-13,2014,4.0
1,2014,3,2014-01-20,2014,29.0
2,2014,4,2014-01-27,2014,47.0
3,2014,5,2014-02-03,2014,36.0
4,2014,6,2014-02-10,2014,42.0


In [111]:
# Prepare dataframe for Prophet
df_national_prophet = df_national.rename(columns={'Date': 'ds', 'total_cases': 'y'})
df_national_prophet['ds'] = pd.to_datetime(df_national_prophet['ds'])
df_national_prophet = df_national_prophet.sort_values('ds')
print("DataFrame prepared for Prophet - Head:")
print(df_national_prophet.head())
print("")
print("DataFrame prepared for Prophet - Tail:")
print(df_national_prophet.tail())

DataFrame prepared for Prophet - Head:
   Year  Week         ds  Epi_Year     y
0  2014     2 2014-01-13      2014   4.0
1  2014     3 2014-01-20      2014  29.0
2  2014     4 2014-01-27      2014  47.0
3  2014     5 2014-02-03      2014  36.0
4  2014     6 2014-02-10      2014  42.0

DataFrame prepared for Prophet - Tail:
     Year  Week         ds  Epi_Year     y
568  2024    48 2024-11-25      2024  33.0
569  2024    49 2024-12-02      2024  49.0
570  2024    50 2024-12-09      2024  38.0
571  2024    51 2024-12-16      2024  32.0
572  2024    52 2024-12-23      2024  36.0


In [112]:
df_states_original = pd.read_csv('../data/processed/data_processed_v3_NoAcum_Total.csv')
df_states_sex_original = pd.read_csv('../data/processed/data_processed_v3_NoAcum_MF.csv')
states_list = df_states_original['Entity'].unique().tolist()
try:
    states_list.remove('National')
except ValueError:
    pass

In [113]:
# Prepare dataframes for each state
dfs_states = {}
for state in states_list:
    df_state = df_states_original[df_states_original['Entity'] == state].copy()
    df_state = df_state.groupby('Date')['total_cases'].sum().reset_index()
    df_state = df_state.rename(columns={'Date': 'ds', 'total_cases': 'y'})
    df_state['ds'] = pd.to_datetime(df_state['ds'])
    df_state = df_state.sort_values('ds')
    dfs_states[state] = df_state

print("Prepared DataFrames for each state:")
for i, (state, df) in enumerate(dfs_states.items()):
    print(f"State: {state}")
    print(df.head())
    print("")
    if i == 5: 
        break

Prepared DataFrames for each state:
State: Aguascalientes
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: Baja California
          ds    y
0 2014-01-13  0.0
1 2014-01-20  6.0
2 2014-01-27  1.0
3 2014-02-03  3.0
4 2014-02-10  4.0

State: Baja California Sur
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: Campeche
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  1.0
3 2014-02-03  1.0
4 2014-02-10  3.0

State: Chiapas
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  1.0
3 2014-02-03  1.0
4 2014-02-10  2.0

State: Chihuahua
          ds    y
0 2014-01-13  0.0
1 2014-01-20  2.0
2 2014-01-27  2.0
3 2014-02-03  2.0
4 2014-02-10  0.0



In [114]:
# Prepare dataframes for each state and sex
dfs_states_sex = {}
for state in states_list:
    for sex in ['M', 'F']: # 'M' = Male, 'F' = Female
        df_ss = df_states_sex_original[df_states_sex_original['Entity'] == state].copy()
        df_ss = df_ss.groupby('Date')[sex].sum().reset_index()
        df_ss = df_ss.rename(columns={'Date': 'ds', sex: 'y'})
        df_ss['ds'] = pd.to_datetime(df_ss['ds'])
        df_ss = df_ss.sort_values('ds')
        dfs_states_sex[(state, sex)] = df_ss
   

print("Prepared DataFrames for each state:")
for i, (state, df) in enumerate(dfs_states_sex.items()):
    print(f"State: {state}")
    print(df.head())
    print("")
    if i == 5:
        break

Prepared DataFrames for each state:
State: ('Aguascalientes', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Aguascalientes', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Baja California', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  1.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  1.0

State: ('Baja California', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  5.0
2 2014-01-27  1.0
3 2014-02-03  3.0
4 2014-02-10  3.0

State: ('Baja California Sur', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Baja California Sur', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0



In [115]:
# Holidays definition
years = range(2014, 2030) 
holidays_list = []
for year in years:
    holidays_list.append({'ds': f'{year}-12-25', 'holiday': 'Navidad'})
    holidays_list.append({'ds': f'{year}-01-01', 'holiday': 'Ano_Nuevo'})
    easter_date = dateutil.easter.easter(year)
    holidays_list.append({'ds': easter_date.strftime('%Y-%m-%d'), 'holiday': 'Semana_Santa'})
    friday_before_easter = easter_date - pd.Timedelta(days=2)
    holidays_list.append({'ds': friday_before_easter.strftime('%Y-%m-%d'), 'holiday': 'Viernes_Santo'})

# create dataframe once and set Semana_Santa window
holidays = pd.DataFrame(holidays_list)
holidays.loc[holidays['holiday'] == 'Semana_Santa', 'lower_window'] = -3
holidays.loc[holidays['holiday'] == 'Semana_Santa', 'upper_window'] = 3

In [116]:
# Covid-19 Indicator

start_covid = pd.Timestamp('2020-03-01')
end_covid = pd.Timestamp('2022-12-31') 
def covid_indicator(date):
    if start_covid <= date <= end_covid:
        return 1
    else:
        return 0


df_national_prophet['covid_dummy'] = df_national_prophet['ds'].apply(covid_indicator)
for state, df_state in dfs_states.items():
    df_state['covid_dummy'] = df_state['ds'].apply(covid_indicator)
for key, df_ss in dfs_states_sex.items():
    df_ss['covid_dummy'] = df_ss['ds'].apply(covid_indicator)

print("DataFrames with Covid-19 indicator added:")
print("National DataFrame - Head:")
print(df_national_prophet.head())
print("")
print("State DataFrame Example - Head:")
example_state = list(dfs_states.keys())[0]
print(f"State: {example_state}")
print(dfs_states[example_state].head())
print("")
print("State and Sex DataFrame Example - Head:")
example_key = list(dfs_states_sex.keys())[0]
print(f"State and Sex   : {example_key}")
print(dfs_states_sex[example_key].head())


DataFrames with Covid-19 indicator added:
National DataFrame - Head:
   Year  Week         ds  Epi_Year     y  covid_dummy
0  2014     2 2014-01-13      2014   4.0            0
1  2014     3 2014-01-20      2014  29.0            0
2  2014     4 2014-01-27      2014  47.0            0
3  2014     5 2014-02-03      2014  36.0            0
4  2014     6 2014-02-10      2014  42.0            0

State DataFrame Example - Head:
State: Aguascalientes
          ds    y  covid_dummy
0 2014-01-13  0.0            0
1 2014-01-20  0.0            0
2 2014-01-27  0.0            0
3 2014-02-03  0.0            0
4 2014-02-10  0.0            0

State and Sex DataFrame Example - Head:
State and Sex   : ('Aguascalientes', 'M')
          ds    y  covid_dummy
0 2014-01-13  0.0            0
1 2014-01-20  0.0            0
2 2014-01-27  0.0            0
3 2014-02-03  0.0            0
4 2014-02-10  0.0            0


In [117]:
# Configure and Train Prophet for National Data

model_national = Prophet(yearly_seasonality=True, 
                         weekly_seasonality=False, 
                         daily_seasonality=False, 
                         seasonality_mode='multiplicative', 
                         holidays=holidays, 
                         changepoint_prior_scale=0.1, # It lets the model detect trend changes moderately
                         seasonality_prior_scale=5.0 # It gives us more flexibility in fitting seasonal patterns but with regularization
)
model_national.add_regressor('covid_dummy')
model_national.fit(df_national_prophet)

16:23:55 - cmdstanpy - INFO - Chain [1] start processing
16:23:55 - cmdstanpy - INFO - Chain [1] done processing


In [118]:
# Create dataframe for future predictions (5 years ahead)
future_national = model_national.make_future_dataframe(periods=260, freq='W-MON') # Weekly on Mondays
future_national['covid_dummy'] = future_national['ds'].apply(covid_indicator)
forecast_national = model_national.predict(future_national)

print("Forecast for National Data - Tail:")
print(forecast_national[['ds', 'yhat','yhat_lower','yhat_upper']].tail(5))

Forecast for National Data - Tail:
            ds       yhat  yhat_lower  yhat_upper
826 2029-11-19  29.548149 -132.907432  179.561455
827 2029-11-26  26.858353 -130.295924  185.210891
828 2029-12-03  27.432981 -127.989325  182.122020
829 2029-12-10  29.677390 -125.406158  184.661294
830 2029-12-17  28.822117 -121.512353  188.996073


In [119]:
# Configure and fit Prophet model for state-level data
models_states = {}
forecasts_states = {}
for state, df_state in dfs_states.items():
    m = Prophet(
    yearly_seasonality=True, 
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode='multiplicative',
    holidays=holidays,
    changepoint_prior_scale=0.1,
    seasonality_prior_scale=5.0
    )
    m.add_regressor('covid_dummy')
    m.fit(df_state)

    models_states[state] = m
    future = m.make_future_dataframe(periods=260, freq='W-MON')
    future['covid_dummy'] = future['ds'].apply(covid_indicator)
    forecast = m.predict(future)
    forecasts_states[state] = forecast

16:23:55 - cmdstanpy - INFO - Chain [1] start processing
16:23:55 - cmdstanpy - INFO - Chain [1] done processing
16:23:56 - cmdstanpy - INFO - Chain [1] start processing
16:23:56 - cmdstanpy - INFO - Chain [1] done processing
16:23:56 - cmdstanpy - INFO - Chain [1] start processing
16:23:56 - cmdstanpy - INFO - Chain [1] done processing
16:23:57 - cmdstanpy - INFO - Chain [1] start processing
16:23:57 - cmdstanpy - INFO - Chain [1] done processing
16:23:57 - cmdstanpy - INFO - Chain [1] start processing
16:23:57 - cmdstanpy - INFO - Chain [1] done processing
16:23:58 - cmdstanpy - INFO - Chain [1] start processing
16:23:58 - cmdstanpy - INFO - Chain [1] done processing
16:23:59 - cmdstanpy - INFO - Chain [1] start processing
16:23:59 - cmdstanpy - INFO - Chain [1] done processing
16:24:00 - cmdstanpy - INFO - Chain [1] start processing
16:24:00 - cmdstanpy - INFO - Chain [1] done processing
16:24:01 - cmdstanpy - INFO - Chain [1] start processing
16:24:01 - cmdstanpy - INFO - Chain [1]

In [120]:
# Create and train Prophet models for each state and sex
models_states_sex = {}
forecasts_states_sex = {}
for (state, sex), df_ss in dfs_states_sex.items():
    m = Prophet(
    yearly_seasonality=True, 
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode='multiplicative',
    holidays=holidays,
    changepoint_prior_scale=0.1,
    seasonality_prior_scale=5.0
    )
    m.add_regressor('covid_dummy')
    m.fit(df_ss)
    models_states_sex[(state, sex)] = m
    future = m.make_future_dataframe(periods=260, freq='W-MON')
    future['covid_dummy'] = future['ds'].apply(covid_indicator)
    forecast = m.predict(future)
    forecasts_states_sex[(state, sex)] = forecast

16:24:12 - cmdstanpy - INFO - Chain [1] start processing
16:24:12 - cmdstanpy - INFO - Chain [1] done processing
16:24:12 - cmdstanpy - INFO - Chain [1] start processing
16:24:13 - cmdstanpy - INFO - Chain [1] done processing
16:24:13 - cmdstanpy - INFO - Chain [1] start processing
16:24:13 - cmdstanpy - INFO - Chain [1] done processing
16:24:13 - cmdstanpy - INFO - Chain [1] start processing
16:24:13 - cmdstanpy - INFO - Chain [1] done processing
16:24:14 - cmdstanpy - INFO - Chain [1] start processing
16:24:14 - cmdstanpy - INFO - Chain [1] done processing
16:24:14 - cmdstanpy - INFO - Chain [1] start processing
16:24:15 - cmdstanpy - INFO - Chain [1] done processing
16:24:15 - cmdstanpy - INFO - Chain [1] start processing
16:24:15 - cmdstanpy - INFO - Chain [1] done processing
16:24:15 - cmdstanpy - INFO - Chain [1] start processing
16:24:15 - cmdstanpy - INFO - Chain [1] done processing
16:24:16 - cmdstanpy - INFO - Chain [1] start processing
16:24:16 - cmdstanpy - INFO - Chain [1]

In [ ]:
# Structure forecasts into a better format that will be used later for reconciliation 
H = 260 # Forecast horizon in weeks
y_base = []

yhat_national = forecast_national['yhat'].iloc[-260:].values
yhat_states = {}
for state in states_list:
    yhat_states[state] = forecasts_states[state]['yhat'].iloc[-260:].values

yhat_states_sex = {}
for state in states_list:
    for sex in ['M', 'F']:
        yhat_states_sex[(state, sex)] = forecasts_states_sex[(state, sex)]['yhat'].iloc[-260:].values

# National level
y_base.append(yhat_national)
# State level
for state in states_list:
    y_base.append(yhat_states[state])
# State and sex level
for state in states_list:
    y_base.append(yhat_states_sex[(state, 'M')])
    y_base.append(yhat_states_sex[(state, 'F')])

Y_base = np.vstack(y_base)  
print(Y_base.shape)  # Shape: (97, 260) -> 1 National + 32 States + 32 States * 2 Genders

(97, 260)


In [ ]:
# Reconciliation with MinT
n = Y_base.shape[0]  # Total number of series
m = 64  # Number of series bottom
# Construct the summing matrix S
S = np.zeros((n, m))
# National level
S[0, :] = 1

# State level
for i, state in enumerate(states_list):
    col_M = 2*(i-1) # Males of state i 
    col_F = 2*(i-1) + 1 # Females of state i
    S[i, col_M] = 1
    S[i, col_F] = 1

# State and sex level
for j in range(m):
    S[1 + len(states_list)+j, j] = 1 # Bottom level series

print("Summing matrix S shape:", S.shape)


Summing matrix S shape: (97, 64)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [138]:
# Proyection matrix for OLS reconciliation
inv_Sst = np.linalg.inv(S.T @ S) 
G_ols = inv_Sst @ S.T
Y_recon_bottom = G_ols @ Y_base
Y_reconciled = S @ Y_recon_bottom # Forecast Matrix (97 x H) 

# Now we can verify that the reconciliation worked
# Check National level
national_reconciliated_week1 = Y_reconciled[0, 0]
sum_states_week1 = np.sum(Y_reconciled[1:1+len(states_list), 0])
print(f"National reconciliated week 1: {national_reconciliated_week1}")
print(f"Sum states week 1: {sum_states_week1}")

print("")
# Check State level
for i, state in enumerate(states_list):
    state_reconciliated_week1 = Y_reconciled[1 + i, 0]
    col_M = 2*i
    col_F = 2*i + 1
    sum_state_reconciliated_week1 = Y_reconciled[1 + len(states_list) + col_M, 0] + Y_reconciled[1 + len(states_list) + col_F, 0]
    print(f"--- State: {state}")
    print(f"\tState reconciliated week 1: {state_reconciliated_week1}")
    print(f"\tSum state reconciliated week 1: {sum_state_reconciliated_week1}")
    if i == 5:  # Just check first 6 states
        break

National reconciliated week 1: 25.730400772052114
Sum states week 1: 25.64138021460853

--- State: Aguascalientes
	State reconciliated week 1: 0.3737081159706862
	Sum state reconciliated week 1: 0.3737081159706862
--- State: Baja California
	State reconciliated week 1: 0.7569475957974903
	Sum state reconciliated week 1: 0.7569475957974903
--- State: Baja California Sur
	State reconciliated week 1: 0.12963150716596786
	Sum state reconciliated week 1: 0.12963150716596786
--- State: Campeche
	State reconciliated week 1: 0.09488922753251938
	Sum state reconciliated week 1: 0.09488922753251938
--- State: Chiapas
	State reconciliated week 1: 0.7276687157920506
	Sum state reconciliated week 1: 0.7276687157920506
--- State: Chihuahua
	State reconciliated week 1: 2.4607421054176606
	Sum state reconciliated week 1: 2.4607421054176606
